## Foundry 에이전트에서 MCP 도구 사용하기

이전 실습에서 테스트한 것처럼, AzureAIAgentClient를 사용하면 쉽게 Foundry Portal에서 유지되는 에이전트를 생성할 수 있습니다.   
그리고 원격 Model Context Protocol (MCP) 서버에서 호스팅되는 도구를 에이전트에 연결하여 Foundry 에이전트의 기능을 확장할 수 있습니다.

><div style="font-weight: 600; color: #835B00; margin-bottom: 4px;">⚠️ 주의사항</div>
>
>현재 AzureAIAgentClient를 사용하여 에이전트를 생성, 관리하는 것은 Foundry Portal(Classic)에서만 지원됩니다.   
>신규 포탈인 Foundry Portal(Preview)에서는 아직 지원되지 않습니다.

🧩 MCP 도구에 대한 기본 실습은 🔌[**에이전트를 MCP 도구로 활용(사용 및 제공)**](MAF/7.Agent-as-MCP-tool.ipynb) 페이지를 참고하시기 바랍니다.


### AzureAIAgentClient에 MCP 도구 연결하기

이번 실습은 Microsoft Learn MCP 서버를 사용하여, 에이전트에 MCP 도구를 연결하는 방법을 다룹니다.

<div style="background: #EFF6FC; border-left: 4px solid #0078D4; border-radius: 0 8px 8px 0; padding: 14px 20px; margin: 14px 0; color: #323130;">
  <p style="margin: 0; line-height: 1.7;">Microsoft Learn MCP는 별도의 인증이나 API 키 없이 사용할 수 있는 <strong>공개 MCP 서버</strong>입니다. <br />누구나 자유롭게 Microsoft 공식 기술 문서를 검색하는 에이전트를 구성할 수 있습니다.</p>
</div>


In [ ]:
# 코드 예시 ...

async def basic_foundry_mcp_example():
    """Basic example of Foundry agent with hosted MCP tools."""
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(credential=credential) as client,
    ):
        # Create a hosted MCP tool using the client method
        learn_mcp = client.get_mcp_tool(
            name="Microsoft Learn MCP",
            url="https://learn.microsoft.com/api/mcp",
            approval_mode="never",  # 승인 없이 MCP 도구 자동 실행
        )

        # Create agent with hosted MCP tool
        agent = client.as_agent(
            name="MicrosoftLearnAgent", 
            instructions="당신은 Microsoft Learn 관련 내용은 MCP를 활용해서 질문에 답변합니다. MCP를 사용할 수 없는 질문은 모른다고 답변해야 합니다.",
            tools=learn_mcp,
        )

        # Simple query without approval workflow
        session = agent.create_session()
        result = await agent.run(
            "Azure AI Agent 문서에서 MCP 도구 호출과 관련된 내용을 요약해 주세요.",
            session=session,
        )
        print(result.text)


if __name__ == "__main__":
    asyncio.run(basic_foundry_mcp_example())

# 현재 폴더에는 완성된 agent_tool.py 파일이 제공됩니다. 
# 실행을 위해서는 우선 터미널을 열고, 가상환경을 활성화 한 후에 다음 명령어를 실행합니다.
#
# .\.venv\Scripts\Activate.ps1; 
# python .\MAF\agent_tool.py

결과는 다음과 같습니다:  
MCP 호출 이력을 포함하여 에이전트가 Microsoft Learn MCP 도구를 사용하여 기술 문서를 검색하는 것을 볼 수 있습니다. 다만, 이것만 봐서는 실제로 MCP 서버에서 제공하는 도구를 호출했는지 확신할 수 없습니다. 그래서 다음 실습은 MCP 호출 내역도 출력해 봅니다.

![hosted-mcp-001.png](images/hosted-mcp-001.png)

다음과 같이 코드를 변경하여 MCP 호출 내역도 출력해 봅니다 

In [ ]:
# 코드 예시 ... 상위 코드는 기존과 동일합니다

        # Simple query without approval workflow
        session = agent.create_session()
        result = await agent.run(
            "Azure AI Agent 문서에서 MCP 도구 호출과 관련된 내용을 요약해 주세요.",
            session=session,
        )

        # MCP 호출 이력 출력
        print("=" * 60)
        print("📋 MCP 호출 이력")
        print("=" * 60)
        thread_id = session.service_session_id
        agents_client = client.agents_client
        mcp_call_count = 0
        runs = agents_client.runs.list(thread_id=thread_id)
        async for run in runs:
            steps = agents_client.run_steps.list(thread_id=thread_id, run_id=run.id)
            async for step in steps:
                if step.type == "tool_calls" and hasattr(step, "step_details"):
                    for tc in step.step_details.tool_calls:
                        if tc.type == "mcp":
                            mcp_call_count += 1
                            print(f"\n[호출 #{mcp_call_count}]")
                            print(f"  서버  : {tc.server_label}")
                            print(f"  도구  : {tc.name}")
                            print(f"  인수  : {tc.arguments}")
                            print(f"  결과  : {str(tc.output)[:300]}...")
        if mcp_call_count == 0:
            print("  (MCP 호출 없음)")
        print("=" * 60)
        print("\n💬 최종 응답:")
        print(result.text)


if __name__ == "__main__":
    asyncio.run(basic_foundry_mcp_example())

# 현재 폴더에는 완성된 agent_tool2.py 파일이 제공됩니다. 
# 실행을 위해서는 우선 터미널을 열고, 가상환경을 활성화 한 후에 다음 명령어를 실행합니다.
#
# .\.venv\Scripts\Activate.ps1; 
# python .\MAF\agent_tool.py


해해ㅔ헤헤ㅔㅎ

ㅇ;ㅔ헤ㅐㅔㅁㅇ


![image.png](./images/hosted-mcp-002.png)

RuntimeError: asyncio.run() cannot be called from a running event loop